In [1]:
from functions import *

C:\KBData\01_PROJECT\00_GIT\ELTE\ELTE_AI\final_project\03_preprocess\functions.py:28: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [2]:
NUM_EP = 2
LR = 2e-5
WEIGHT_DECAY = .01
BATCH_SIZE=8
GRAD_ACCUM=1

In [3]:
LR_SCHEDULE = 'cosine'
LR_WARMUP_STEPS = 1000

In [4]:
MODEL_CP = "../02_dependencies/hubert-base-cc"
MODEL_NAME = "hupunct-v02f-01"

In [5]:
train_text_file_path = '../data/dataset_v2/train_data.txt'
fp_train = open(train_text_file_path, 'r', encoding='utf-8')
fp_eval = open(train_text_file_path.replace('train', 'eval'), 'r', encoding='utf-8')

In [6]:
bert_max_tokens = 512

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CP, model_max_len=bert_max_tokens)

In [8]:
ds_train = Dataset.from_generator(token_label_generator,
                                  gen_kwargs={'file_pointer': fp_train, 'mappings': mappings})

# ds_train_sequenced = ds_train.map(sequence_example,
#                                   batched=True,
#                                   remove_columns=ds_train.column_names,
#                                   load_from_cache_file=True,
#                                   fn_kwargs={'increment': INCREMENT, 'sequence_length': SEQUENCE_LENGTH}
#                                   )

ds_train = ds_train.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ds_train.column_names,
    load_from_cache_file=True,
    fn_kwargs={'tokenizer': tokenizer}
)

Found cached dataset generator (C:/Users/ext-gyenist/.cache/huggingface/datasets/generator/default-1adf31276521924c/0.0.0)
Loading cached processed dataset at C:\Users\ext-gyenist\.cache\huggingface\datasets\generator\default-1adf31276521924c\0.0.0\cache-33a04c6e17beb13f.arrow


In [9]:
ds_eval = Dataset.from_generator(token_label_generator, gen_kwargs={'file_pointer': fp_eval, 'mappings': mappings})



# ds_eval_sequenced = ds_eval.map(sequence_example,
#                                 batched=True,
#                                 remove_columns=ds_eval.column_names,
#                                 load_from_cache_file=True,
#                                 fn_kwargs={'increment': INCREMENT, 'sequence_length': SEQUENCE_LENGTH}
#                                 )

ds_eval = ds_eval.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=ds_eval.column_names,
    load_from_cache_file=True,
    fn_kwargs={'tokenizer': tokenizer}
)

Found cached dataset generator (C:/Users/ext-gyenist/.cache/huggingface/datasets/generator/default-cdf822eac05b784c/0.0.0)
Loading cached processed dataset at C:\Users\ext-gyenist\.cache\huggingface\datasets\generator\default-cdf822eac05b784c\0.0.0\cache-0d14521c213716ef.arrow


In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer,
                                                   padding='max_length',
                                                   max_length=bert_max_tokens,
                                                   )

In [11]:
model = AutoModelForTokenClassification.from_pretrained(
    MODEL_CP,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at ../02_dependencies/hubert-base-cc were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not init

In [12]:
args = TrainingArguments(
    report_to=['wandb'],
    output_dir=f'D:/00_DATA/02_ELTE/MODELS/{MODEL_NAME}',
    run_name=MODEL_NAME,
    logging_strategy='steps',
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_steps=400//GRAD_ACCUM,
    eval_steps=1600//GRAD_ACCUM,
    save_steps=1600//GRAD_ACCUM,
    load_best_model_at_end=True,
    save_total_limit=5,
    metric_for_best_model='f1',

    lr_scheduler_type=LR_SCHEDULE,
    warmup_steps=LR_WARMUP_STEPS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRAD_ACCUM,
    eval_accumulation_steps=GRAD_ACCUM,
    learning_rate=LR,

    weight_decay=WEIGHT_DECAY,
    num_train_epochs=NUM_EP,
)

In [13]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_eval,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [14]:
trainer.train(resume_from_checkpoint=True)

trainer.save_model(f"../04_models/{MODEL_NAME}")

C:\KBApps\conda\envs\torch2\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: tamgyen. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
251200,0.185300,0.176291,0.873013,0.864088,0.868528,0.938277
252800,0.188500,0.176897,0.874267,0.861638,0.867906,0.938032
254400,0.186700,0.176779,0.870136,0.866569,0.868349,0.938133
256000,0.185900,0.177216,0.873357,0.863880,0.868593,0.938379


In [ ]:
# steps = 1130 eta: 6:40

In [ ]:
# 836196